In [1]:
import numpy as np

alpha = 0.1       
gamma = 0.99      
epsilon = 1.0     
epsilon_decay = 0.995
min_epsilon = 0.01


In [ ]:
def initialize_q_tables(n_agents, state_space, action_space):
    q_tables = [np.zeros((state_space, action_space)) for _ in range(n_agents)]

    return q_tables
    

def choose_action_with_positional_message(agent_idx, state, q_tables, action_space, n_agents, message_dim):
    if np.random.rand() < epsilon:
        action = np.random.choice(action_space) 
    else:
        action = np.argmax(q_tables[agent_idx][state])

    # rotary position embedding?? or not here 
    content_vector = ...
    positional_encoding = ...
    message = (content_vector, positional_encoding)

    return action, message


def route_messages(messages):
    routed_messages = env._route_messages(messages)
    return routed_messages

def update_q_value(q_table, state, action, reward, next_state):
    best_next_action = np.argmax(q_table[next_state])
    q_table[state, action] = q_table[state, action] + alpha * (
        reward + gamma * q_table[next_state, best_next_action] - q_table[state, action]
    )

def train_multi_agent_with_position(env, q_tables, episodes, n_agents, message_dim):
    global epsilon
    for episode in range(episodes):
        states = env.reset()
        done = False
        while not done:
            actions = []
            messages = []
            for agent_idx in range(n_agents):
                action, message = choose_action_with_positional_message(
                    agent_idx, states[agent_idx], q_tables, env.action_space[agent_idx].n, n_agents, message_dim
                )
                actions.append(action)
                messages.append(message)

            received_actions = tuple((actions[i], messages[i]) for i in range(n_agents))
            
            next_states, rewards, done, _ = env.step(actions)
            routed_messages = env._route_messages(received_actions)  # Corrected format for `_route_messages`

            for agent_idx in range(n_agents):
                update_q_value(
                    q_tables[agent_idx], states[agent_idx], actions[agent_idx], rewards[agent_idx], next_states[agent_idx]
                )
            
            states = next_states
        
        epsilon = max(min_epsilon, epsilon * epsilon_decay)

env = UNOCommunicationEnv(n_agents=..., message_dim=...)
state_space = ...
action_space = ...

q_tables = initialize_q_tables(n_agents=4, state_space=state_space, action_space=action_space)

train_multi_agent_with_position(env, q_tables, episodes=1000, n_agents=4, message_dim=5)
